# Inference with a pretrained model

In [1]:
from model_components.k_means import  CustomKMeans
import numpy as np
from model_components.CNN_model import Cplx_CustomCNN_1D
from model_components.Attention_Layer import  AttentionLayer
from model_components.classification import CustomClassifierModel
import tensorflow as tf

import re
import numpy as np
import pandas as pd

## DATA

In [2]:
# Load the data
df = pd.read_csv('Prepro_Data/post_processed_data_small20.csv')

def extract_multiple_arrays(string_repr):
    # Remove letters, parentheses, spaces, and unwanted sequences
    string_repr = re.sub('[a-df-zA-DF-Z\(\)\s]', '', string_repr)
    string_repr = re.sub(r',=32', '', string_repr)

    # Find all matches of arrays within the string
    arrays = re.findall(r'\[.*?\]', string_repr)

    # Convert each found array string into a NumPy array
    #np_arrays = [np.array(re.findall(r'[+-]?\d+(?:\.\d+)?', array), dtype=float) for array in arrays]
    np_arrays = [np.array(re.findall(r'[+-]?\d+(?:\.\d+)?(?:e[+-]?\d+)?', array), dtype=float) for array in arrays]
    return np_arrays

df['pixels_value'] = df['pixels_value'].apply(extract_multiple_arrays)
# pixels float to int
print(df.head())

   Unnamed: 0  segment_id  polygon_id  class_id  \
0           0           0           0         0   
1           1           1           0         0   
2           2           2           0         0   
3           3           3           0         0   
4           4           4           0         0   

                                              pixels  \
0  [[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [1, 0...   
1  [[0, 5], [0, 6], [0, 7], [0, 8], [1, 5], [1, 6...   
2  [[0, 9], [0, 10], [0, 11], [0, 12], [1, 9], [1...   
3  [[0, 13], [0, 14], [0, 15], [0, 16], [1, 13], ...   
4  [[0, 17], [0, 18], [0, 19], [1, 16], [1, 17], ...   

                                    Perimeter Pixels  \
0  [[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [4, 4...   
1  [[0, 5], [1, 5], [2, 5], [3, 5], [4, 5], [4, 8...   
2  [[0, 9], [1, 9], [2, 9], [3, 9], [4, 9], [4, 1...   
3  [[0, 13], [1, 13], [2, 13], [3, 13], [4, 13], ...   
4  [[0, 17], [1, 16], [2, 16], [3, 16], [4, 17], ...   

                   

## ALGORITHM

In [3]:
# K-means
stacked_arrays = []

# Stack the arrays for each cell
for i, cell_pixels in enumerate(df['pixels_value']):
    stacked_array = np.vstack(cell_pixels)
    stacked_arrays.append(stacked_array)
        
n_clusters=2
custom_kmeans = CustomKMeans(n_clusters=n_clusters)

# Fit the model to the data and get the cluster centers
clustered_data = []
clustered_labels = []
for i in range (len(stacked_arrays)):
    custom_kmeans.fit(stacked_arrays[i])
    clustered_data.append(custom_kmeans.get_cluster_centers())
    clustered_labels.append(custom_kmeans.get_cluster_labels())
    
clusters = np.array(clustered_data)

/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/si

In [4]:
class Custom_Model(tf.keras.Model):
    '''
    This class defines the model architecture.
    '''
    
    def __init__(self):
        super(Custom_Model, self).__init__()
        self.enc = Cplx_CustomCNN_1D()
        self.attn = AttentionLayer()
        self.classifier = CustomClassifierModel(num_classes=8, fc_units=64) 

    def call(self, inputs):
        input_list = tf.unstack(inputs,axis=1) # from tensor to list
        intermediate = [self.enc(input) for input in input_list]
        intermediate = tf.stack(intermediate,axis=1)        
        emb, alphas = self.attn(intermediate)
        
        return self.classifier(emb)
    
    def get_alphas(self, inputs):
        input_list = tf.unstack(inputs,axis=1) # from tensor to list
        intermediate = [self.enc(input) for input in input_list]
        intermediate = tf.stack(intermediate,axis=1)        
        _, alphas = self.attn(intermediate)
        
        return alphas


### Initialize the model

In [5]:
model = Custom_Model()
input_shape = (2, 73, 10)  

# Create a dummy input tensor for model initialization
dummy_input = tf.random.normal((1,) + input_shape)

# Pass the dummy input through the model to initialize the layers
model(dummy_input)

2023-12-21 21:34:41.143285: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-12-21 21:34:41.143489: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-12-21 21:34:41.143493: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-12-21 21:34:41.144252: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-21 21:34:41.145160: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(1, 8), dtype=float32, numpy=
array([[0.12578271, 0.12545593, 0.1223575 , 0.12644641, 0.12201456,
        0.12761815, 0.12366813, 0.1266566 ]], dtype=float32)>

In [6]:
# Load the weights
SAVE_PATH = 'saved_models/'

model.load_weights(SAVE_PATH + 'model_weights_100_epochs.h5')

## RESULTS

In [7]:
import math

predictions = model.predict(clusters)

df_inferring = df.iloc[:, 1:6]
# Get the class with the highest probability
df_inferring['class_id'] = np.argmax(predictions, axis=1) + 1
df_inferring['class_id'] = df_inferring['class_id'].astype('int8')

# Pixels' cluster
# Assuming you have 'clustered_labels' available
df_inferring['pixels_cluster'] = clustered_labels

# Alphas
alphas = model.get_alphas(clusters)
alphas = np.array(alphas)
df_inferring['alphas'] = alphas.tolist()

# Function to round to the nearest hundredth
def round_to_hundredth(lst):
    return [math.ceil(num * 100) / 100 for num in lst]

df_inferring['alphas'] = df_inferring['alphas'].apply(round_to_hundredth)

# Function to replace the cluster labels with the corresponding alpha values, ideal to make a heatmap
def replace_values(row):
    return [row['alphas'][val] for val in row['pixels_cluster']]

df_inferring['heatmap'] = df_inferring.apply(replace_values, axis=1)
df_inferring.head()

df_inferring.head()

2023-12-21 21:34:53.717257: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


111/111 [==============================] - 3s 20ms/step


,segment_id,polygon_id,class_id,pixels,Perimeter Pixels,pixels_cluster,alphas,heatmap
0,0,0,6,"[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [1, 0...","[[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [4, 4...","[1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, ...","[0.34, 0.67]","[0.67, 0.67, 0.67, 0.34, 0.34, 0.34, 0.34, 0.3..."
1,1,0,6,"[[0, 5], [0, 6], [0, 7], [0, 8], [1, 5], [1, 6...","[[0, 5], [1, 5], [2, 5], [3, 5], [4, 5], [4, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[0.26, 0.75]","[0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.2..."
2,2,0,6,"[[0, 9], [0, 10], [0, 11], [0, 12], [1, 9], [1...","[[0, 9], [1, 9], [2, 9], [3, 9], [4, 9], [4, 1...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.64, 0.37]","[0.37, 0.37, 0.37, 0.37, 0.64, 0.64, 0.64, 0.6..."
3,3,0,5,"[[0, 13], [0, 14], [0, 15], [0, 16], [1, 13], ...","[[0, 13], [1, 13], [2, 13], [3, 13], [4, 13], ...","[0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, ...","[0.55, 0.46]","[0.55, 0.55, 0.46, 0.46, 0.55, 0.55, 0.46, 0.5..."
4,4,0,5,"[[0, 17], [0, 18], [0, 19], [1, 16], [1, 17], ...","[[0, 17], [1, 16], [2, 16], [3, 16], [4, 17], ...","[1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, ...","[0.65, 0.36]","[0.36, 0.36, 0.36, 0.36, 0.65, 0.65, 0.36, 0.3..."


In [ ]:
# save df_inferring as csv
df_inferring.to_csv('Prepro_Data/Inference_results_20.txt', index=False)